### Context
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

### Content
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

### Goal
Identify fraudulent credit card transactions in this highly imbalanced dataset using oversampling via SMOTE.

In [2]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix



from IPython.display import display # Allows the use of display() for DataFrames
%matplotlib inline

# Load the dataset
data0 = pd.read_csv('creditcard.csv')

#prices = data['MEDV']
#features = data.drop('MEDV', axis = 1)
    
# Success
print("Test dataset has {} data points with {} variables each.".format(*data0.shape))

Test dataset has 284807 data points with 31 variables each.


### The data were already checked for missing values in the EDA notebook. Next, I rescale the 'Amount' feature, drop the 'Time' feature, and reassemble the rescaled feature matrix.

In [3]:
# Check how imbalanced the data is
print('non-fraudulent transactions', round(data0['Class'].value_counts()[0]/len(data0) * 100,2), '%')
print('fraudulent transactions', round(data0['Class'].value_counts()[1]/len(data0) * 100,2), '%')

non-fraudulent transactions 99.83 %
fraudulent transactions 0.17 %


In [4]:
# Rescale 'Amount', drop 'Time' as it is not relevant for predicting fraud
scaler = StandardScaler()
data0['Amount_norm'] = scaler.fit_transform(data0['Amount'].values.reshape(-1, 1))

# dataframe of the V-features only:
v_data = data0.drop(['Amount_norm', 'Time', 'Amount'], axis=1)

# New dataframe with the normalized Amount_norm as a first column, for convenience
data = pd.concat([data0['Amount_norm'], v_data], axis=1)
data.head()

,Amount_norm,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,0.244964,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.342475,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,1.160686,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,0.140534,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,-0.073403,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


### Separate the predictors and the dependent variable 'Class'. Split the data into training and test sets.

In [5]:
# Separate the predictors from the dependent variable ('Class'):
X = data.iloc[:, data.columns != 'Class']
y = data.iloc[:, data.columns == 'Class']

In [23]:
# Split the data into training and test datasets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
# Check how imbalanced the training set is: it roughly retained the imbalance ratio from before splitting
print('non-fraudulent transactions', round(y_train['Class'].value_counts()[0]/len(y_train) * 100,2), '%')
print('fraudulent transactions', round(y_train['Class'].value_counts()[1]/len(y_train) * 100,2), '%')

non-fraudulent transactions 99.82 %
fraudulent transactions 0.18 %


### Here, I chose to increase the number of points in the minority class using SMOTE. Import the method, and fit the training data (do NOT touch the test data!):

In [24]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)
np.bincount(y_train)

array([227440, 227440])

### Fit a logistic regression classifier to the (now balanced) training data:

In [10]:
logreg = LogisticRegression(random_state=1, solver='liblinear')
logreg.fit(X_train, y_train)
y_pred=logreg.predict(X_test)

In [11]:
# How did we do?
print('Accuracy = ', logreg.score(X_test, y_test))
print('Recall = ', recall_score(y_test, logreg.predict(X_test)))

Accuracy =  0.9787402127734279
Recall =  0.8850574712643678


### I was aiming for high recall. Let's look at the metrics: 

In [12]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[55674  1201]
 [   10    77]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56875
           1       0.06      0.89      0.11        87

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.55     56962
weighted avg       1.00      0.98      0.99     56962



### Great! The recall score is 0.89.

### Next, let's try a different classifier.

In [13]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [14]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[56864    11]
 [   18    69]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56875
           1       0.86      0.79      0.83        87

    accuracy                           1.00     56962
   macro avg       0.93      0.90      0.91     56962
weighted avg       1.00      1.00      1.00     56962



In [53]:
type(X_test)

pandas.core.frame.DataFrame

### The random forest classifier resulted in a really high precision and reduced recall score compared to the logistic regression classifier. It was kind of slow, too - training it took > 4 minutes!

### Let's try xgboost:

In [34]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBClassifier  

xclas = XGBClassifier()  # and for classifier  
xclas.fit(X_train, y_train)  




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [56]:

y_pred = xclas.predict(X_test.values)  

In [57]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[56253   622]
 [   11    76]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56875
           1       0.11      0.87      0.19        87

    accuracy                           0.99     56962
   macro avg       0.55      0.93      0.59     56962
weighted avg       1.00      0.99      0.99     56962

